In [ ]:
pip install matplotlib

In [ ]:
pip install hmmlearn==0.2.7

In [ ]:
pip install python_speech_features

In [ ]:
import warnings
warnings.filterwarnings('ignore')
import os
import librosa
from hmmlearn import hmm
import numpy as np
from sklearn.cluster import KMeans
from sklearn.metrics import confusion_matrix
from scipy.io import wavfile
from scipy.io.wavfile import read
import wave
from python_speech_features import mfcc
from mlxtend.plotting import plot_confusion_matrix
from scipy.fftpack import dct
import matplotlib.pyplot as plt
from mpl_toolkits import mplot3d
from matplotlib.ticker import MaxNLocator
from matplotlib import cm
import plotly.graph_objects as go
import pandas as pd
from scipy.interpolate import griddata

In [ ]:
   #解壓縮

In [ ]:
trainDir = 'true_train/' #檔案路徑

In [ ]:
testDir = 'true_test/'

In [ ]:
#@title 端點偵測 特徵萃取 畫圖
import scipy.fftpack as fftpack
from scipy.fftpack.realtransforms import dct
import matplotlib.pyplot as plt
import wave
import numpy as np
import sklearn
def test1(path,flagl):
    #path = "0.wav"
    f = wave.open(path,"r")
    # read the wave's format infomation,and return a tuple
    params = f.getparams()
    # get the info
    nchannels, sampwidth, framerate, nframes = params[:4]
    # Reads and returns nframes of audio, as a string of bytes. 
    str_data = f.readframes(nframes)
    # close the stream
    f.close()
    # turn the wave's data to array
    wave_data = np.frombuffer(str_data, dtype = np.short)
    # for the data is stereo,and format is LRLRLR...
    # shape the array to n*2(-1 means fit the y coordinate)
    wave_data.shape = -1, 2#(資料自行決定音訊長度,雙通道/多通道(須回原DATASET確定))
    # transpose the data
    wave_data = wave_data.T
    original_wave = wave_data[0]#取單通道做處理
    #print(wave_data)

    Frame_len = 256
    idx = 0
    flag = 0
    data = []
    while(flag == 0):
        tmp = 0.0
        for i in range(int(idx),int(idx + Frame_len) ):
            if idx + Frame_len >= len(original_wave):
                flag = 1
                tmp += 0
            else:
                tmp += original_wave[i] ** 2              
        data.append(tmp)
        idx = idx + Frame_len    

    #original_wave = np.array(wave_data) 不太懂這行
    #print(original_wave)
    k = 3  #可調整
    max_frame = max(data)
    threshold = 0.075 * max_frame + sum(data[0:k:]) / k
    #print(threshold)
    

    EPD_wave = []
    start_idxs = []
    end_idxs = []
    EPD_wave_start_idxs = []
    EPD_wave_end_idxs = []
    i = 0
    while(i < len(data)):
        endure = 1 #能接受幾個沒聲音的音框
        # 找出EPD_wave的start_point
        while(i < len(data)):
            if data[i] > threshold:
                start_point = i * Frame_len#實際上是在 *128 的INDEX 因為是這裡的DATA[i]指的是音框
                start_idxs.append(start_point)
                break
            else:
                i += 1
            i += 1

        # 找出EPD_wave的end_point
        while(i < len(data)):
            if data[i] < threshold:
                if endure != 0:
                    endure -= 1
                    i += 1
                    continue
                else:
                    end_point = i * Frame_len
                    end_idxs.append(end_point)
                    i += 1
                    break
            else:
                i += 1
        #print(start_point,end_point)
        # if len(end_idxs) and len(start_idxs) and end_idxs[-1] == start_idxs[-1]:
        #     end_idxs.pop()
        #     start_idxs.pop()  
        #print(EPD_wave)
        #print(original_wave[start_point:end_point])
        if(len(EPD_wave_start_idxs)==0):
          EPD_wave_start_idxs.append(0)
          EPD_wave_end_idxs.append(end_point-start_point)
        else:
          EPD_wave_start_idxs.append(EPD_wave_end_idxs[-1])#加大一點方便看出來 不太行因為會導致後面的一起被 例如第一個只有100 第十個則有1000
          EPD_wave_end_idxs.append(EPD_wave_start_idxs[-1]+end_point-start_point)
        EPD_wave=np.concatenate([EPD_wave,original_wave[start_point:end_point]])
    #print(EPD_wave)
        

    
    #這裡加上 儲存start end point 就好
    start_t = np.array(start_idxs)
    end_t = np.array(end_idxs)  
    EPD_wave_start_t=np.array(EPD_wave_start_idxs)
    EPD_wave_end_t=np.array(EPD_wave_end_idxs)
    n = 256
    m = 0.5

    Frame_len = n
    overlap = Frame_len * m
    idx = 0
    flag = 0
    frame = []
    while(flag == 0):
        tmp = []
        '''如果這段音框截取的位置沒有超過聲音的長度就直接截取
            否則就慢慢複製，不夠的部分補0'''
        if idx + Frame_len < len(EPD_wave):
            tmp = np.copy(EPD_wave[int(idx) : int(idx+Frame_len)]) # get frame
        else:
            for i in range(int(idx),int(idx+Frame_len) ):
                if idx + Frame_len >= len(EPD_wave):
                    flag = 1
                    tmp.append(0)
                else:
                    tmp.append(EPD_wave[i])
        tmp = np.array(tmp)                    
        frame.append(tmp)
        idx = idx + overlap


    EPD_wave = np.array(EPD_wave)
    EPD_wave = EPD_wave.reshape(-1,)

    # Transform list to np.array
    frame = np.array(frame) 

    pre_emphasis = []
    for i in range(frame.shape[0]):
        tmp = np.append(frame[i][0], frame[i][1:] - 0.95 * frame[i][:-1])
        tmp = np.array(tmp)
        pre_emphasis.append(tmp)
    pre_emphasis = np.array(pre_emphasis)    

    hamming_wave = np.hamming(pre_emphasis.shape[1]) * pre_emphasis

    NFFT = 512
    nfilt = 13
    num_ceps = 12
    mag_frames = np.absolute(np.fft.rfft(hamming_wave, NFFT))   # fft的幅度
    pow_frames = ((1.0 / NFFT) * ((mag_frames) ** 2))  # 功率譜
    low_freq_mel = 0
    high_freq_mel = (2595 * np.log10(1 + (framerate / 2) / 700))  # Transfom Hz to Mel
    mel_points = np.linspace(low_freq_mel, high_freq_mel, nfilt + 2)  # 使Mel 刻度間距相等
    hz_points = (700 * (10**(mel_points / 2595) - 1))  # 將Mel轉換為Hz
    bin = np.floor((NFFT + 1) * hz_points / framerate)

    fbank = np.zeros((nfilt, int(np.floor(NFFT / 2 + 1))))
    for m in range(1, nfilt + 1):
        f_m_minus = int(bin[m - 1])   # 左
        f_m = int(bin[m])             # 中
        f_m_plus = int(bin[m + 1])    # 右

        for k in range(f_m_minus, f_m):
            fbank[m - 1, k] = (k - bin[m - 1]) / (bin[m] - bin[m - 1])
        for k in range(f_m, f_m_plus):
            fbank[m - 1, k] = (bin[m + 1] - k) / (bin[m + 1] - bin[m])
    filter_banks = np.dot(pow_frames, fbank.T)
    filter_banks = np.where(filter_banks == 0, np.finfo(float).eps, filter_banks)  # 数值穩定性
    filter_banks = 20 * np.log10(filter_banks)  # dB
    mfcc = dct(filter_banks, type=2, axis=1, norm='ortho')[:, 1 : (num_ceps+1)]
    #np.savetxt('mfcc_data.txt',mfcc)
    
    # MFCC standardization
    mfcc = sklearn.preprocessing.StandardScaler().fit_transform(mfcc)

    # Compute delta MFCC and delta-delta MFCC
    delta_mfcc = librosa.feature.delta(mfcc)
    delta2_mfcc = librosa.feature.delta(mfcc, order=2)

    # Concatenate MFCC, delta MFCC and delta-delta MFCC
    all_mfcc = np.concatenate([mfcc, delta_mfcc, delta2_mfcc], axis=0)

    if flagl == 1:
      plt.figure()
      plt.suptitle("before & after of EPD", fontsize=20)
      plt.subplot(2, 1, 1)
      plt.plot(original_wave)
      for s, e in zip(start_t, end_t):
        #print(s,e)
        plt.axvline(x=s, c='#2ca02c') #綠色
        plt.axvline(x=e, c='#d62728')  #紅色
      plt.subplot(2, 1, 2)
      plt.plot(EPD_wave)
      for s, e in zip(EPD_wave_start_t, EPD_wave_end_t):
        #print(s,e)
        plt.axvline(x=s, c='#2ca02c') #綠色
        #plt.axvline(x=e, c='#d62728')  #紅色
      
      plt.show()
    return all_mfcc

In [ ]:
fileList = os.listdir(trainDir)
trainDataSet = {}
flagl = 0
for fileName in fileList:
    tmp = fileName.split('.')[0]#第幾筆資料         #跟檔名用.分割
    label = tmp.split('_')[1]#這裡是實際的label          #跟檔名用_分割
    #Edata ,framerate = EPD(trainDir+fileName)
    #feature = extract_mfcc(trainDir+fileName,Edata,framerate) 
    feature = test1(trainDir+fileName,flagl)
    print(label)
    if label not in trainDataSet.keys():
        trainDataSet[label] = []
        trainDataSet[label].append(feature)
    else:
        exist_feature = trainDataSet[label]
        exist_feature.append(feature)
        trainDataSet[label] = exist_feature
print("Finish prepare the training data")


In [ ]:
big_trainData = trainDataSet['0']+trainDataSet['1']+trainDataSet['2']+trainDataSet['3']

big_trainData = np.vstack(big_trainData)

In [ ]:
#有經過離三角濾波器的轉換 Y(M)會比原本的資料來得少

In [ ]:
#print(len(trainDataSet['3']))

In [ ]:
def getrate(clus,state,trainDataSet,big_trainData,testDataSet,fileList):
    
  start_clu =clus #初始群心數
  n_clu=clus  #最大群心數

  start_states_num = state #隱藏狀態從2開始計算
  max_states_num = state
  

  recognition_rate_array = {}
  a = 0 
  n_clu_array = {}
  b = 0
  states_array = {}
  d = 0
    
  for c in range(start_clu,n_clu+1):
    kmeans = KMeans(n_clusters=c)#應該是c
    kmeans.fit(big_trainData)
    centers = kmeans.cluster_centers_
  
    for g in range(start_states_num,max_states_num+1):
      HMM_Models = {}
      states_num = g   #設定有多少隱藏狀態
      for i in trainDataSet.keys():               #迭代次數 #使用演算法     #準確度的容忍
        model = hmm.MultinomialHMM(n_components=states_num,n_iter=20,algorithm='viterbi', tol=0.01)
        trainData = trainDataSet[i]
        trainData = np.vstack(trainData)
        trainData_label = []
        for j in range(len(trainData)):
          dic_min = np.linalg.norm(trainData[j]-centers[0])
          label = 0
          for k in range(len(centers)):
              if np.linalg.norm(trainData[j]-centers[k])<dic_min:
                  dic_min = np.linalg.norm(trainData[j]-centers[k])
                  label = k
          trainData_label.append(label)
              
        trainData_label.append(len(centers)-1)  #以群的編號代換特徵值
        trainData_label = np.array([trainData_label])
        model.fit(trainData_label)
        HMM_Models[i] = model

      print("Finish training of the DHMM models")



      score_cnt = 0
      true = []
      pred = []
      for i in testDataSet.keys():
        feature = testDataSet[i]
        for h in range(len(feature)):
          testData_label = []
          for j in range(len(feature[h])):
              dic_min = np.linalg.norm(feature[h][j]- centers[0])
              label = 0
              for k in range(len(centers)):    
                  if np.linalg.norm(feature[h][j]-centers[k])<dic_min:
                      dic_min = np.linalg.norm(feature[h][j]-centers[k])
                      label = k
              testData_label.append(label)
          scoreList = {}
          testData_label = np.array([testData_label])
          for model_label in HMM_Models.keys():
              model = HMM_Models[model_label]
              score = model.score(testData_label)
              scoreList[model_label] = score
          predict = max(scoreList, key=scoreList.get)
          if i==predict:
              score_cnt+=1
          #print("True label is ",i,"and predict result label is ",predict)
          true.append(int(i[i.find('(')+1]))
          pred.append(int(predict))

      recognition_rate = 100.0 * score_cnt / (len(fileList))
      recognition_rate = round(recognition_rate,2)
      print("Recognition rate is", recognition_rate, "%")
      n_clu_array[b] = c
      recognition_rate_array[a] = [recognition_rate]
      states_array[d]=g
      a+=1
      b+=1
      d+=1
      true = np.array(true)
      pred = np.array(pred)

      # 计算每个类别的召回率
      recall_per_class = recall_score(true, pred, average=None)

      # 计算 UAR（平均召回率）
      uar = np.mean(recall_per_class)

      #print("UAR is", uar)
      return uar

In [ ]:
fileList = os.listdir(testDir)
testDataSet = {}
flag2=1
for fileName in fileList:
    tmp = fileName.split('.')[0]
    label = tmp.split('_')[1]
    feature = test1(testDir+fileName,flag2)
    flag2=0
    if label not in testDataSet.keys():
      testDataSet[label] = []
      testDataSet[label].append(feature)
    else:
      exist_feature = testDataSet[label]
      exist_feature.append(feature)
      testDataSet[label] = exist_feature
print("Finish prepare the testing data")

### **3d圖**

In [ ]:
start_clu = 1 #初始群心數
n_clu= 100  #最大群心數

start_states_num = 1 #隱藏狀態從2開始計算
max_states_num = 100
#固定是1
cluj=10   #群心一次跳幾個
statej=10  #隱藏狀態一次跳幾個

all_array = [[0 for x in range(max_states_num+statej)] for y in range(n_clu+cluj)]  
               #columns                 #rows

recognition_rate_array = {}
a = 0 
n_clu_array = {}
b = 0
states_array = {}
d = 0



fileList = os.listdir(testDir)
for c in range(start_clu,n_clu+1,cluj):
  kmeans = KMeans(n_clusters=c)#應該是c
  kmeans.fit(big_trainData)
  centers = kmeans.cluster_centers_
 
  for g in range(start_states_num,max_states_num+1,statej):
    print(c,g)
    HMM_Models = {}
    states_num = g   #設定有多少隱藏狀態
    for i in trainDataSet.keys():               #迭代次數 #使用演算法     #準確度的容忍
      model = hmm.MultinomialHMM(n_components=states_num,n_iter=20,algorithm='viterbi', tol=0.01)
      trainData = trainDataSet[i]

      trainData = np.vstack(trainData)
      trainData_label = []
      for j in range(len(trainData)):
        dic_min = np.linalg.norm(trainData[j]-centers[0])
        label = 0
        for k in range(len(centers)):
            if np.linalg.norm(trainData[j]-centers[k])<dic_min:
                dic_min = np.linalg.norm(trainData[j]-centers[k])
                label = k
        trainData_label.append(label)
            
      trainData_label.append(len(centers)-1)  #以群的編號代換特徵值
      trainData_label = np.array([trainData_label])
      model.fit(trainData_label)
      HMM_Models[i] = model

    print("Finish training of the DHMM models")

    score_cnt = 0
    true = []
    pred = []
    for i in testDataSet.keys():
      feature = testDataSet[i]
      for h in range(len(feature)):
        testData_label = []
        for j in range(len(feature[h])):
            dic_min = np.linalg.norm(feature[h][j]- centers[0])
            label = 0
            for k in range(len(centers)):    
                if np.linalg.norm(feature[h][j]-centers[k])<dic_min:
                    dic_min = np.linalg.norm(feature[h][j]-centers[k])
                    label = k
            testData_label.append(label)
        scoreList = {}
        testData_label = np.array([testData_label])
        for model_label in HMM_Models.keys():
            model = HMM_Models[model_label]
            score = model.score(testData_label)
            scoreList[model_label] = score
        predict = max(scoreList, key=scoreList.get)
        if i==predict:
            score_cnt+=1
        #print("True label is ",i,"and predict result label is ",predict)
        true.append(int(i[i.find('(')+1]))
        pred.append(int(predict))

    true = np.array(true)
    pred = np.array(pred)
    # 计算每个类别的召回率
    recall_per_class = recall_score(true, pred, average=None)

    # 计算 UAR（平均召回率）
    uar = np.mean(recall_per_class)
    recognition_rate = 100.0 * score_cnt / (len(fileList))
    recognition_rate = round(recognition_rate,2)
    print("Recognition rate is", recognition_rate, "%")
    n_clu_array[b]=c
    recognition_rate_array[a] = [uar]
    states_array[d]=g
    a+=1
    b+=1
    d+=1
    all_array[c][g]=uar
    print(all_array[c][g])
    print("UAR is", uar)
print("ok")



In [ ]:
recognition_rate_tenford=[]
n_clu_tenford=[]
states_tenford=[]
#print(len(recognition_rate_array))
for i in range(len(recognition_rate_array)):
  recognition_rate_tenford.append(recognition_rate_array[i][0])
  n_clu_tenford.append(n_clu_array[i])
  # print(i)
  states_tenford.append(states_array[i])   
import copy 
m = recognition_rate_tenford
t = copy.deepcopy(m)
# 求m个最大的数值及其索引
max_number = []
max_num_index_list = []
for _ in range(5):
    number = max(t)
    index = t.index(number)
    t[index] = 0
    max_number.append(number)
    max_num_index_list.append(index)
t = []
print(max_number)
print(max_num_index_list)


In [ ]:
tenford="true_tenfordset/"
flag=0
BigTrainDataSet={}
for i in range(1,11):
    Train = {}
    FileList = os.listdir(tenford+str(i-1)+"/")
    for fileName in FileList:
      tmp = fileName.split('.')[0]#第幾筆資料         #跟檔名用.分割
      label = tmp.split('_')[1]#這裡是實際的label          #跟檔名用_分割
      feature = test1(tenford+str(i-1)+"/"+fileName,flag)    
      if label not in Train.keys():
          Train[label] = []
          Train[label].append(feature)
      else:
          exist_feature = Train[label]
          exist_feature.append(feature)
          Train[label] = exist_feature
    BigTrainDataSet[i]=Train
    print(i)
    

In [ ]:
print(BigTrainDataSet)

In [ ]:
arr=[[0]*(10) for i in range(5)]
flag=0
#print(n_clu_array[max_num_index_list[0]])
for i in range(1,11):
    TrainDataSet = {}
    TestDataSet = {}
    for tt in range(4):
        TrainDataSet[str(tt)]=[]
        TestDataSet[str(tt)]=[]
    for k in range(1,11):
        if(k==i):
            continue
        TrainDataSet['0']+=BigTrainDataSet[k]['0']
        TrainDataSet['1']+=BigTrainDataSet[k]['1']
        TrainDataSet['2']+=BigTrainDataSet[k]['2']
        TrainDataSet['3']+=BigTrainDataSet[k]['3']
    TrainData = TrainDataSet['0']+TrainDataSet['1']+TrainDataSet['2']+TrainDataSet['3']
    TrainData = np.vstack(TrainData)
    TestDataSet['0']+=BigTrainDataSet[i]['0']
    TestDataSet['1']+=BigTrainDataSet[i]['1']
    TestDataSet['2']+=BigTrainDataSet[i]['2']
    TestDataSet['3']+=BigTrainDataSet[i]['3']
      #print("Finish prepare the testing data")
    FILELIST=os.listdir(tenford+str(i-1)+"/")
    for j in range(5):#計得要改回11
    #print(now)
      arr[j][i-1]=getrate(n_clu_tenford[max_num_index_list[j]],states_tenford[max_num_index_list[j]],TrainDataSet,TrainData,TestDataSet,FILELIST)
    print(arr)
  #記得要改 i,j

In [ ]:
taveragerate=[0]*(5)
for i in range(5):
  now=0
  for j in range(1,11):#計得要改回11
    now+=arr[i][j-1]
  taveragerate[i]=now/10
best=taveragerate.index(max(taveragerate))
print(arr)
print(taveragerate)
print("最佳的隱藏狀態和分群數量為"+str(n_clu_tenford[max_num_index_list[best]])+" "+str(states_tenford[max_num_index_list[best]]))
print(states_tenford)

In [ ]:
for i in range(len(max_num_index_list)):
    print(n_clu_tenford[max_num_index_list[i]],states_tenford[max_num_index_list[i]])